## Installing dependencies

In [78]:
!pip3 install -U flwr-nightly
!pip3 install -U flwr-nightly[simulation]
!pip install torchdiffeq
!pip install fastdtw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Download the dataset**

In [79]:
# !rm -r Pems_Dataset/
!git clone https://github.com/ebagirma/Pems_Dataset.git
%cd Pems_Dataset
!ls

fatal: destination path 'Pems_Dataset' already exists and is not an empty directory.
/content/Pems_Dataset
models	PEMS04	pems04_dtw_distance.npy  pems04_spatial_distance.npy  README.md


In [80]:
from collections import OrderedDict
from typing import List, Tuple

from flwr.common import Metrics

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

import pickle
import numpy as np
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
import torch

from Pems_Dataset.models.model import ODEGCN
# from utils import MyDataset, read_data, get_normalized_adj
# from eval import masked_mae_np, masked_mape_np, masked_rmse_np

from torch.utils.data import DataLoader, random_split

import ray
import flwr as fl



import os
import csv

from fastdtw import fastdtw
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch

In [81]:
if torch.cuda. is_available():
    print("Using Cuda!")
    DEVICE = torch.device("cuda")
else:
    print("Using CPU")
    DEVICE = torch.device("cpu")

Using Cuda!


# Load arguments

In [82]:
class Args:
    def __init__(self):
        self.remote = False
        self.num_gpu = 0
        self.global_epochs = 5
        self.local_epoch = 10
        self.batch_size = 16
        self.batch = 16
        self.frac = 0.1
        self.num_users = 100
        self.filename = 'pems04'
        self.train_ratio = 0.6
        self.valid_ratio = 0.2
        self.his_length = 12
        self.pred_length = 12
        self.sigma1 = 0.1
        self.sigma2 = 10
        self.thres1 = 0.6
        self.thres2 = 0.5
        self.lr = 2e-3
        self.log = False

args = Args()

In [83]:
files = {
    'pems03': ['PEMS03/pems03.npz', 'PEMS03/distance.csv'],
    'pems04': ['PEMS04/PEMS04.npz', 'PEMS04/distance.csv'],
    'pems07': ['PEMS07/pems07.npz', 'PEMS07/distance.csv'],
    'pems08': ['PEMS08/pems08.npz', 'PEMS08/distance.csv'],
    'pemsbay': ['PEMSBAY/pems_bay.npz', 'PEMSBAY/distance.csv'],
    'pemsD7M': ['PeMSD7M/PeMSD7M.npz', 'PeMSD7M/distance.csv'],
    'pemsD7L': ['PeMSD7L/PeMSD7L.npz', 'PeMSD7L/distance.csv']
}

In [84]:
%cd /content/

def read_data(args):
    """read data, generate spatial adjacency matrix and semantic adjacency matrix by dtw

    Args:
        sigma1: float, default=0.1, sigma for the semantic matrix
        sigma2: float, default=10, sigma for the spatial matrix
        thres1: float, default=0.6, the threshold for the semantic matrix
        thres2: float, default=0.5, the threshold for the spatial matrix

    Returns:
        data: tensor, T * N * 1
        dtw_matrix: array, semantic adjacency matrix
        sp_matrix: array, spatial adjacency matrix
    """
    filename = args.filename
    file = files[filename]
    filepath = "./Pems_Dataset/"
    if args.remote:
        filepath = './Pems_Dataset/'


        
    data = np.load(filepath + file[0])['data']
    # PEMS04 == shape: (16992, 307, 3)    feature: flow,occupy,speed
    # PEMSD7M == shape: (12672, 228, 1)
    # PEMSD7L == shape: (12672, 1026, 1)
    num_node = data.shape[1]
    mean_value = np.mean(data, axis=(0, 1)).reshape(1, 1, -1)
    std_value = np.std(data, axis=(0, 1)).reshape(1, 1, -1)
    data = (data - mean_value) / std_value
    mean_value = mean_value.reshape(-1)[0]
    std_value = std_value.reshape(-1)[0]

    if not os.path.exists(f'Pems_Dataset/{filename}_dtw_distance.npy'):
        data_mean = np.mean([data[:, :, 0][24*12*i: 24*12*(i+1)] for i in range(data.shape[0]//(24*12))], axis=0)
        data_mean = data_mean.squeeze().T 
        dtw_distance = np.zeros((num_node, num_node))
        for i in tqdm(range(num_node)):
            for j in range(i, num_node):
                dtw_distance[i][j] = fastdtw(data_mean[i], data_mean[j], radius=6)[0]
        for i in range(num_node):
            for j in range(i):
                dtw_distance[i][j] = dtw_distance[j][i]
        np.save(f'data/{filename}_dtw_distance.npy', dtw_distance)

    dist_matrix = np.load(f'Pems_Dataset/{filename}_dtw_distance.npy')

    mean = np.mean(dist_matrix)
    std = np.std(dist_matrix)
    dist_matrix = (dist_matrix - mean) / std
    sigma = args.sigma1
    dist_matrix = np.exp(-dist_matrix ** 2 / sigma ** 2)
    dtw_matrix = np.zeros_like(dist_matrix)
    dtw_matrix[dist_matrix > args.thres1] = 1

    # # use continuous semantic matrix
    # if not os.path.exists(f'data/{filename}_dtw_c_matrix.npy'):
    #     dist_matrix = np.load(f'data/{filename}_dtw_distance.npy')
    #     # normalization
    #     std = np.std(dist_matrix[dist_matrix != np.float('inf')])
    #     mean = np.mean(dist_matrix[dist_matrix != np.float('inf')])
    #     dist_matrix = (dist_matrix - mean) / std
    #     sigma = 0.1
    #     dtw_matrix = np.exp(- dist_matrix**2 / sigma**2)
    #     dtw_matrix[dtw_matrix < 0.5] = 0 
    #     np.save(f'data/{filename}_dtw_c_matrix.npy', dtw_matrix)
    # dtw_matrix = np.load(f'data/{filename}_dtw_c_matrix.npy')
    
    # use continuous spatial matrix
    if not os.path.exists(f'Pems_Dataset/{filename}_spatial_distance.npy'):
        with open(filepath + file[1], 'r') as fp:
            dist_matrix = np.zeros((num_node, num_node)) + np.float('inf')
            file = csv.reader(fp)
            for line in file:
                break
            for line in file:
                start = int(line[0])
                end = int(line[1])
                dist_matrix[start][end] = float(line[2])
                dist_matrix[end][start] = float(line[2])
            np.save(f'Pems_Dataset/{filename}_spatial_distance.npy', dist_matrix)



    dist_matrix = np.load(f'Pems_Dataset/{filename}_spatial_distance.npy')
    # normalization
    std = np.std(dist_matrix[dist_matrix != np.inf])
    mean = np.mean(dist_matrix[dist_matrix != np.inf])
    dist_matrix = (dist_matrix - mean) / std
    sigma = args.sigma2
    sp_matrix = np.exp(- dist_matrix**2 / sigma**2)
    sp_matrix[sp_matrix < args.thres2] = 0 
 

    print(f'average degree of spatial graph is {np.sum(sp_matrix > 0)/2/num_node}')
    print(f'average degree of semantic graph is {np.sum(dtw_matrix > 0)/2/num_node}')
    return torch.from_numpy(data.astype(np.float32)), mean_value, std_value, dtw_matrix, sp_matrix


def get_normalized_adj(A):
    """
    Returns a tensor, the degree normalized adjacency matrix.
    """
    alpha = 0.8
    D = np.array(np.sum(A, axis=1)).reshape((-1,))
    D[D <= 10e-5] = 10e-5    # Prevent infs
    diag = np.reciprocal(np.sqrt(D))
    A_wave = np.multiply(np.multiply(diag.reshape((-1, 1)), A),
                         diag.reshape((1, -1)))
    A_reg = alpha / 2 * (np.eye(A.shape[0]) + A_wave)
    return torch.from_numpy(A_reg.astype(np.float32))


class MyDataset(Dataset):
    def __init__(self, data, split_start, split_end, his_length, pred_length):
        split_start = int(split_start)
        split_end = int(split_end)
        self.data = data[split_start: split_end]
        self.his_length = his_length
        self.pred_length = pred_length
    
    def __getitem__(self, index):
        x = self.data[index: index + self.his_length].permute(1, 0, 2)
        y = self.data[index + self.his_length: index + self.his_length + self.pred_length][:, :, 0].permute(1, 0)
        return torch.Tensor(x), torch.Tensor(y)
    def __len__(self):
        return self.data.shape[0] - self.his_length - self.pred_length + 1


def generate_dataset(data, args):
    """
    Args:
        data: input dataset, shape like T * N
        batch_size: int 
        train_ratio: float, the ratio of the dataset for training
        his_length: the input length of time series for prediction
        pred_length: the target length of time series of prediction

    Returns:
        train_dataloader: torch tensor, shape like batch * N * his_length * features
        test_dataloader: torch tensor, shape like batch * N * pred_length * features
    """
    batch_size = args.batch_size
    train_ratio = args.train_ratio
    valid_ratio = args.valid_ratio
    his_length = args.his_length
    pred_length = args.pred_length
    train_dataset = MyDataset(data, 0, data.shape[0] * train_ratio, his_length, pred_length)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MyDataset(data, data.shape[0]*train_ratio, data.shape[0]*(train_ratio+valid_ratio), his_length, pred_length)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = MyDataset(data, data.shape[0]*(train_ratio+valid_ratio), data.shape[0], his_length, pred_length)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, valid_dataloader, test_dataloader


/content


In [85]:
from torch.utils.data import DataLoader, random_split

class MyDataset(Dataset):
    def __init__(self, data, split_start, split_end, his_length, pred_length):
        split_start = int(split_start)
        split_end = int(split_end)
        self.data = data[split_start: split_end]
        self.his_length = his_length
        self.pred_length = pred_length
    
    def __getitem__(self, index):
        x = self.data[index: index + self.his_length].permute(1, 0, 2)
        y = self.data[index + self.his_length: index + self.his_length + self.pred_length][:, :, 0].permute(1, 0)
        return torch.Tensor(x), torch.Tensor(y)
    
    def __len__(self):
        return self.data.shape[0] - self.his_length - self.pred_length + 1




In [86]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    #print('works till here', len(state_dict))
    net.load_state_dict(state_dict, strict=True)

In [87]:
def mask_np(array, null_val):
    if np.isnan(null_val):
        return (~np.isnan(null_val)).astype('float32')
    else:
        return np.not_equal(array, null_val).astype('float32')


def masked_mape_np(y_true, y_pred, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        mask = mask_np(y_true, null_val)
        mask /= mask.mean()
        mape = np.abs((y_pred - y_true) / y_true)
        mape = np.nan_to_num(mask * mape)
        return np.mean(mape) * 100


def masked_rmse_np(y_true, y_pred, null_val=np.nan):
    mask = mask_np(y_true, null_val)
    mask /= mask.mean()
    mse = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(np.nan_to_num(mask * mse)))


def masked_mae_np(y_true, y_pred, null_val=np.nan):
    mask = mask_np(y_true, null_val)
    mask /= mask.mean()
    mae = np.abs(y_true - y_pred)
    return np.mean(np.nan_to_num(mask * mae))


In [88]:
def train(loader, model, optimizer, criterion, epochs, scheduler, std, mean,device):
    print('training for epochs:', epochs)
    epoch_loss = []
    model.train()
    for epoch in range(1, epochs+1):
        print('epoch#:', epoch)
        batch_loss = 0
        for idx, (inputs, targets) in enumerate(loader):
            
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            batch_loss += loss.detach().cpu().item() 
        epoch_loss.append(batch_loss)
        loss = batch_loss
       
        train_rmse, train_mae, train_mape = eval(loader, model, std, mean, device)

        if args.log:
            logger.info(f'\n##on train data## loss: {loss}, \n' + 
                        f'##on train data## rmse loss: {train_rmse}, mae loss: {train_mae}, mape loss: {train_mape}\n')
        else:
            print(f'\n##on train data## loss: {loss}, \n' + 
                f'##on train data## rmse loss: {train_rmse}, mae loss: {train_mae}, mape loss: {train_mape}\n')
        
    scheduler.step()

    return sum(epoch_loss)/epochs

In [89]:

@torch.no_grad()
def eval(loader, model, std, mean, device):
    batch_rmse_loss = 0  
    batch_mae_loss = 0
    batch_mape_loss = 0
    print('evaluating')
    model.eval()
    for idx, (inputs, targets) in enumerate(loader):

        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        
        out_unnorm = output.detach().cpu().numpy()*std + mean
        target_unnorm = targets.detach().cpu().numpy()*std + mean

        mae_loss = masked_mae_np(target_unnorm, out_unnorm, 0)
        rmse_loss = masked_rmse_np(target_unnorm, out_unnorm, 0)
        mape_loss = masked_mape_np(target_unnorm, out_unnorm, 0)
        batch_rmse_loss += rmse_loss
        batch_mae_loss += mae_loss
        batch_mape_loss += mape_loss
    print("eval rmse loss: ", batch_rmse_loss/ (idx + 1))

    return batch_rmse_loss / (idx + 1), batch_mae_loss / (idx + 1), batch_mape_loss / (idx + 1)


In [90]:
def generate_dataset(data, args):
    """
    Args:
        data: input dataset, shape like T * N
        batch_size: int 
        train_ratio: float, the ratio of the dataset for training
        his_length: the input length of time series for prediction
        pred_length: the target length of time series of prediction

    Returns:
        train_dataloader: torch tensor, shape like batch * N * his_length * features
        test_dataloader: torch tensor, shape like batch * N * pred_length * features
    """
    batch_size = args.batch_size
    train_ratio = args.train_ratio
    valid_ratio = args.valid_ratio
    his_length = args.his_length
    pred_length = args.pred_length
    train_dataset = MyDataset(data, 0, data.shape[0] * train_ratio, his_length, pred_length)

    valid_dataset = MyDataset(data, data.shape[0]*train_ratio, data.shape[0]*(train_ratio+valid_ratio), his_length, pred_length)

    test_dataset = MyDataset(data, data.shape[0]*(train_ratio+valid_ratio), data.shape[0], his_length, pred_length)

    return train_dataset, valid_dataset, test_dataset


# **Step 2: Federated Learning with Flower**

In [91]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, optimiser, schedular, learning_rate, epochs, loss_function, mean, std ):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.loss_function = loss_function
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.optimiser = torch.optim.AdamW(net.parameters(), lr=self.learning_rate)
        self.scheduler = StepLR(self.optimiser, step_size=50, gamma=0.5)
        self.mean = mean
        self.std = std

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        loss = train(self.trainloader, self.net, self.optimiser ,  self.loss_function, self.epochs , self.scheduler, self.std, self.mean, DEVICE)
        return get_parameters(self.net), len(self.trainloader)*self.trainloader.batch_size, {"train_loss":loss}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        valid_rmse, valid_mae, valid_mape = eval(self.valloader, self.net, self.std, self.mean, DEVICE)
        return valid_rmse, len(self.valloader)*self.valloader.batch_size , {'valid_mape':valid_mape, 'valid_mae':valid_mae, 'valid_rmse':valid_rmse}
    


In [92]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    epochs = 2
    # Load model
    learning_rate = lr
    net = ODEGCN(num_nodes=data.shape[1], 
            num_features=data.shape[2], 
            num_timesteps_input=args.his_length, 
            num_timesteps_output=args.pred_length, 
            A_sp_hat=A_sp_wave, 
            A_se_hat=A_se_wave).to(DEVICE)
    #net.std = std
    #net.mean = mean

    w_glob = net.state_dict()
    
    net.load_state_dict(w_glob, strict=True)
    net(torch.rand(args.batch_size, feature_size, 12, 3).to(DEVICE))
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(cid = cid, net=net, trainloader=trainloader, valloader=valloader, optimiser=None , schedular=None,learning_rate=learning_rate, 
                        epochs=epochs , loss_function= criterion , mean=mean, std=std)

In [93]:
def fit_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    loss = [num_examples * m["train_loss"] for num_examples, m in metrics]
    loss_client = [m["train_loss"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {'client_metrics': {"loss":loss_client}, 'examples':examples,'average_metrics':{'average loss': sum(loss)/sum(examples)}}
    

In [94]:

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used

    valid_rmse = [num_examples * m["valid_rmse"] for num_examples, m in metrics]
    valid_mae = [num_examples * m["valid_mae"] for num_examples, m in metrics]
    valid_mape = [num_examples * m["valid_mape"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    valid_rmse_client = [m["valid_rmse"] for num_examples, m in metrics]
    valid_mae_client = [m["valid_mae"] for num_examples, m in metrics]
    valid_mape_client = [m["valid_mape"] for num_examples, m in metrics]


    # Aggregate and return custom metric (weighted average)
    return {'average_metrics':{'valid_mape':sum(valid_mape)/sum(examples), 'valid_mae':sum(valid_mae)/sum(examples), 'valid_rmse':sum(valid_rmse)/sum(examples)},
            'client_metrics': {'valid_mape':valid_mape_client, 'valid_mae':valid_mae_client, 'valid_rmse':valid_rmse_client, 'examples': examples}}


In [95]:
if args.log:
    logger.add('log_{time}.log')
options = vars(args)
if args.log:
    logger.info(options)
else:
    print(options)

{'remote': False, 'num_gpu': 0, 'global_epochs': 5, 'local_epoch': 10, 'batch_size': 16, 'batch': 16, 'frac': 0.1, 'num_users': 100, 'filename': 'pems04', 'train_ratio': 0.6, 'valid_ratio': 0.2, 'his_length': 12, 'pred_length': 12, 'sigma1': 0.1, 'sigma2': 10, 'thres1': 0.6, 'thres2': 0.5, 'lr': 0.002, 'log': False}


In [96]:
epoch = args.local_epoch
NUM_CLIENTS = 5
!pwd
data, mean, std, dtw_matrix, sp_matrix = read_data(args)
train_loader, valid_loader, test_loader = generate_dataset(data, args)
A_sp_wave = get_normalized_adj(sp_matrix).to(DEVICE)
A_se_wave = get_normalized_adj(dtw_matrix).to(DEVICE)

/content
average degree of spatial graph is 1.1009771986970684
average degree of semantic graph is 6.267100977198697


In [97]:
batch_size = args.batch_size
train_loader = Subset(train_loader, np.arange(1000))
valid_loader = Subset(valid_loader, np.arange(200))
feature_size = train_loader.dataset.data.shape[1]
inds = np.array_split(np.random.randint(len(train_loader), size=len(train_loader)), NUM_CLIENTS)
trainloaders = []
valloaders = []
for idx in inds:
    trainloaders.append(DataLoader(Subset(train_loader, idx), batch_size=batch_size, shuffle=True))


inds = np.array_split(np.random.randint(len(valid_loader), size=len(valid_loader)), NUM_CLIENTS)
for idx in inds:
    valloaders.append(DataLoader(Subset(valid_loader, idx), batch_size=batch_size, shuffle=True))


In [98]:
print("total train loaders:", len(trainloaders))
print("total  val loaders:", len(valloaders))
print("len of single train loader:", len(trainloaders[0]))
print("len of single val loader:", len(valloaders[0]))
print('feature size', feature_size)

total train loaders: 5
total  val loaders: 5
len of single train loader: 13
len of single val loader: 3
feature size 307


In [99]:
lr = args.lr
criterion = nn.SmoothL1Loss()

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=5,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=5,  # Wait until all 10 clients are available
    evaluate_metrics_aggregation_fn =  weighted_average,
    fit_metrics_aggregation_fn = fit_average
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
flower_history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=args.global_epochs),
    strategy=strategy,
    client_resources=client_resources,
)



INFO flwr 2023-04-21 03:53:41,726 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-04-21 03:53:50,497	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-21 03:53:53,645 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7709695182.0, 'object_store_memory': 3854847590.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7709695182.0, 'object_store_memory': 3854847590.0, 'CPU': 2.0}
INFO flwr 2023-04-21 03:53:53,649 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-21 03:53:53,652 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=22903) 2023-04-21 03:53:59.316753

(launch_and_fit pid=23002) training for epochs: 2
(launch_and_fit pid=23002) epoch#: 1
(launch_and_fit pid=23002) evaluating
(launch_and_fit pid=23002) eval rmse loss:  151.5247521033654
(launch_and_fit pid=23002) 
(launch_and_fit pid=23002) ##on train data## loss: 7.670723229646683, 
(launch_and_fit pid=23002) ##on train data## rmse loss: 151.5247521033654, mae loss: 115.90400226299579, mape loss: 186.9636609004094
(launch_and_fit pid=23002) 
(launch_and_fit pid=23002) epoch#: 2
(launch_and_fit pid=23002) evaluating
(launch_and_fit pid=23002) eval rmse loss:  131.12501115065353
(launch_and_fit pid=23002) 
(launch_and_fit pid=23002) ##on train data## loss: 5.608185291290283, 
(launch_and_fit pid=23002) ##on train data## rmse loss: 131.12501115065353, mae loss: 105.06686166616586, mape loss: 181.90289093897894
(launch_and_fit pid=23002) 


(pid=23102) 2023-04-21 03:54:21.283792: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=23102) training for epochs: 2
(launch_and_fit pid=23102) epoch#: 1
(launch_and_fit pid=23102) evaluating
(launch_and_fit pid=23102) eval rmse loss:  162.9861778846154
(launch_and_fit pid=23102) 
(launch_and_fit pid=23102) ##on train data## loss: 8.028758019208908, 
(launch_and_fit pid=23102) ##on train data## rmse loss: 162.9861778846154, mae loss: 125.2217548076923, mape loss: 184.85595675615164
(launch_and_fit pid=23102) 
(launch_and_fit pid=23102) epoch#: 2
(launch_and_fit pid=23102) evaluating
(launch_and_fit pid=23102) eval rmse loss:  151.15193997896634
(launch_and_fit pid=23102) 
(launch_and_fit pid=23102) ##on train data## loss: 5.272757202386856, 
(launch_and_fit pid=23102) ##on train data## rmse loss: 151.15193997896634, mae loss: 112.37478579007663, mape loss: 140.42534232139587
(launch_and_fit pid=23102) 


(pid=23210) 2023-04-21 03:54:36.156034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

(pid=23688) 2023-04-21 03:55:38.756041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=23688) evaluating
(launch_and_evaluate pid=23688) eval rmse loss:  171.62962849934897


(pid=23768) 2023-04-21 03:55:45.477742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=23768) evaluating
(launch_and_evaluate pid=23768) eval rmse loss:  172.91936747233072


(pid=23833) 2023-04-21 03:55:53.331377: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!pwd
with open('flower_history.pkl', 'wb') as f:
    pickle.dump(flower_history, f)

In [ ]:
with open('flower_history.pkl', 'rb') as f:
    history = pickle.load(f)

In [ ]:
# history of each client for training
history.metrics_distributed_fit

In [ ]:
# history of each client for validation
history.metrics_distributed

In [ ]:
x, y = [], []
for average_metrics in history.metrics_distributed['average_metrics']:
    x.append(average_metrics[0])
    y.append(average_metrics[1]['valid_rmse'])


plt.figure(figsize=(12, 6))

plt.plot(x, y, label='Average RMSE in each round')
plt.xlabel('Rounds')
plt.ylabel('RMSE')
plt.legend()
plt.title('Validation Curve for RMSE')
plt.show()


In [ ]:
x =  []
y = [[] for i in history.metrics_distributed['client_metrics'][0][1]['valid_rmse']]
for client_metrics in history.metrics_distributed['client_metrics']:
    x.append(client_metrics[0])
    for idx, client_loss in enumerate(client_metrics[1]['valid_rmse']):
        y[idx].append(client_loss)


plt.figure(figsize=(12, 6))
for client_num, loss_ in enumerate(y):
    plt.plot(x, loss_, label='Validation RMSE of client: '+str(client_num))
plt.xlabel('Rounds')
plt.ylabel('RMSE')
plt.legend()
plt.title('Validation Curve for RMSE (individual client)')
plt.show()

In [ ]:
x, y = [], []
for average_metrics in history.metrics_distributed_fit['average_metrics']:
    x.append(average_metrics[0])
    y.append(average_metrics[1]['average loss'])


plt.figure(figsize=(12, 6))

plt.plot(x, y, label='Average loss in each round')
plt.xlabel('Rounds')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Curve for Loss')
plt.show()

In [ ]:
x =  []
y = [[] for i in history.metrics_distributed_fit['client_metrics'][0][1]['loss']]
for client_metrics in history.metrics_distributed_fit['client_metrics']:
    x.append(client_metrics[0])
    for idx, client_loss in enumerate(client_metrics[1]['loss']):
        y[idx].append(client_loss)

plt.figure(figsize=(12, 6))
for client_num, loss_ in enumerate(y):
    plt.plot(x, loss_, label='Training Loss of client: '+str(client_num))
plt.xlabel('Rounds')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Curve for Loss (individual client)')
plt.show()